### Instructions to fill the Wikibase instance start [here](#Use-dictionaries-to-populate-your-Wikibase)


# To Install the WikiBase follow the RaiseWikibase install instructions: https://github.com/UB-Mannheim/RaiseWikibase


### Requirements:

0. Python 3 (.9 or later, lest you need to order dictionaries on your own); pip3
1. Jupyter notebook installed (obviously :)
2. latest docker version installed
    - for windows: https://docs.docker.com/desktop/windows/install/
    - for macOS: https://docs.docker.com/desktop/mac/install/
    - for linux: you can run the engine without the workaround
3. git (how to: https://git-scm.com/book/en/v2/Getting-Started-Installing-Git)

For Windows user: Anaconda comes with pip, jupyter and git (https://docs.anaconda.com/anaconda/install/windows/)

### use the following lines as an example to create a .env file in the root directory of RaiseWikibase

change Passwords accordingly

### Docker set up  (see https://github.com/UB-Mannheim/RaiseWikibase)



> docker-compose -f docker-compose.yml -f docker-compose.extra.yml up -d --scale wikibase_jobrunner=1


Running

> docker ps

should result in something like this:
https://github.com/UB-Mannheim/RaiseWikibase#:~:text=If%20it%27s%20running%2C%20the%20output%20looks%20like%20this%3A

### ... and clean up

run on line after another in /RaiseWikibase:
> docker-compose down

> sudo rm -rf mediawiki-*  query-service-data/ quickstatements-data/

> docker-compose up -d

this removes the docker container, cleans the necessary storage places and mounts and starts the containers anew.

Note: In the readme to the latest version of RaiseWikibase the second command is replaced by "docker volume prune".

## How it works

the following cells demonstrate how to :
 1. copying items/properties from wikidata
 2. creating items from scratch in RaiseWikibase work
 
Note: IDs of Qualifiers and Properties are erased, when downloaded from wikidata and get a new index. Look at your Wikibase to see, which ones are the right ones to use. You can find a list at: http://localhost/wiki/Special:AllPages?from=&to=&namespace=122&hideredirects=1

### set up a bot to fill your own wikibase

In [1]:
from RaiseWikibase.raiser import create_bot
from RaiseWikibase.settings import Settings
create_bot()
config = Settings()

Bot is created. Metadata is saved to '.config.json'.


In [2]:
config

{'_filename': '.config.json',
 'username': 'MHuber@bot',
 'password': 'tvqvf3mr20k14ati9j7obtcrq7mqhm31',
 'mediawiki_api_url': 'http://localhost:8181/w/api.php',
 'sparql_endpoint_url': 'http://localhost:8989/bigdata/sparql',
 'wikibase_url': 'http://localhost:8181'}

### get specific entity from wikidata

In [10]:
from RaiseWikibase.raiser import batch, page
from RaiseWikibase.datamodel import label, alias, description, snak, claim, entity, namespaces, datatypes
import requests


def get_wd_entity(wid=''):
    """Returns JSON representation of a Wikidata entity for the given WID"""
    # Remove the following keys to avoid a problem with a new Wikibase instance
    remove_keys = ['lastrevid', 'pageid', 'modified', 'title', 'ns']
    #print("1")
    try:
        r = requests.get('https://www.wikidata.org/entity/' + wid + '.json')
        entity = r.json().get('entities').get(wid)
        #print(entity)
        for key in remove_keys:
            entity.pop(key)
            
            #print("popped")
        #entity['claims'] = claim(prop='P1',
         #                        mainsnak=snak(datatype='external-id',
          #                                     value=wid,
           #                                    prop='P1',
            #                                   snaktype='value'),
             #                    qualifiers=[],
              #                   references=[])
    except Exception:
        print('stuff')
        entity = None
    return entity

#add a list of wikidata qualifiers
wids = ['Q386724']
#wids = ['Q5','Q7725634','Q25372','Q1344','Q25379'] # human, work, play, drama, opera 'Q7725634','Q25379','Q25372','Q1344'
items = [get_wd_entity(wid) for wid in wids]
#print(items)

#run batch to upload to your wikibase
#batch('wikibase-item', items)

### get specific property from wikidata

In [ ]:
def property_wd(prop=''):
    """For the given PID of a property in Wikidata returns its 
    simplified JSON represetation (only the label, aliases, description 
    and one claim)"""
    if prop:
        r = requests.get('https://www.wikidata.org/entity/' + prop + '.json').json()
        po = r.get('entities').get(prop)
        p = entity(labels=restrict(po.get('labels')),
                   aliases=restrict(po.get('aliases')),
                   descriptions=restrict(po.get('descriptions')),
                   claims=claim(prop='P1',
                                mainsnak=snak(datatype='external-id',
                                              value=prop,
                                              prop='P1',
                                              snaktype='value')),
                   etype='property',
                   datatype='string')
    else:
        p = None
    return p
#to upload: batch()...

### Use RaiseWikibase "from Scratch"
 - vgl. https://www.wikidata.org/wiki/Wikidata:Glossary/en
 
The following cells showcase how to construct an item/property with Raisewikibase
Note: Properties (P...)and Qualifiers (Q...) are examples you need to adjust.

In [11]:
from RaiseWikibase.datamodel import label, alias, description, snak, claim, entity, statement
from RaiseWikibase.raiser import batch


### create label

In [ ]:
labels = label('de','personitemlabel')

### create alias

In [ ]:
aliases = alias('de','alternativepersonlabel')

### create descriptions

In [44]:
descriptions = description('de','Personenname gefunden in Theaterperiodika des 18. JH')

### create a snak


In [ ]:
#make them human (Q5 = human, P19 = instance of)
mainsnak = snak(datatype='wikibase-item', value="Q5", prop='P19', snaktype='value')

### create a claim

In [ ]:
claims =claim(prop='P19', mainsnak=mainsnak)

### create the entity

In [ ]:
item = entity(labels=labels,descriptions=descriptions,claims=claims,etype='item')

### what an entity object, ready to upload looks like:

In [34]:
#m

{'type': 'item',
 'id': 'Q24500',
 'labels': {'en': {'language': 'en', 'value': 'human'}},
 'aliases': {'en': [{'language': 'en', 'value': 'person'}]},
 'descriptions': {'en': {'language': 'en', 'value': 'a human being'}},
 'claims': {'P1': [{'mainsnak': {'snaktype': 'value',
     'property': 'P1',
     'datavalue': {'value': 'Q5', 'type': 'string'},
     'datatype': 'external-id',
     'hash': 'bc7983b6-ce1b-4a4f-bc60-19691e7f6658'},
    'type': 'statement',
    'rank': 'normal',
    'qualifiers': {'P1': []},
    'qualifiers-order': ['P1'],
    'references': [{'snaks': {'P1': []}, 'snaks-order': ['P1']}],
    'id': 'Q24500$5fa9c6b6-1dcf-4c42-8bfc-dc66808dd4c0'}]}}

### upload
Note: to crate a new item/property use new=True,
to change one, use new=False

In [ ]:
#batch('wikibase-item', [item]], new=True)

### Basic example of how to add and change a property and/or a qualifier
Code is from an example the developer wrote for an issue: https://github.com/UB-Mannheim/RaiseWikibase/issues/15

In [2]:
from RaiseWikibase.datamodel import label, alias, description, snak, claim, entity
from RaiseWikibase.raiser import batch


p = entity(labels=label(value='Wikidata ID'),
           aliases=alias(value=["WID", 'WikidataID']),
           descriptions=description(value="ID of an entity in Wikidata"),
           claims={},
           etype='property',
           datatype='external-id')

#batch('wikibase-property', [p])

e = entity(labels=label(value='human'),
           aliases={},
           descriptions={},
           claims={},
           etype='item')

#batch('wikibase-item', [e])

m = entity(labels=label(value='human'),
           aliases=alias(value=['person']),
           descriptions=description(value='a human being'),
           claims=claim(prop='P1',
                        mainsnak=snak(datatype='external-id',
                                      value='Q5',
                                      prop='P1',
                                      snaktype='value')),
           etype='item')


#important: an ID is mandatory
#m["id"]="Q24500"
#batch('wikibase-item', [m], new=False)

## Export and import Wikibase

complete explanation found here: https://wikibase.consulting/transferring-wikibase-data-between-wikis/

#### Export:

To create a xml dump, connect to wikibase docker container running:

> docker-compose exec -it raisewikibase_wikibase_1 bash

Note: raisewikibase_wikibase_1 is the name of the container

with

> php maintenance/dumpBackup.php --full --quiet > wikibase.xml

you create a xml dump.

To get the dump out of the container, navigate to a suitable location and run:

>docker cp "insertcontainernameorid":/var/www/html/wikibase.xml .
    
Note:
- you can also copy the container id (something like: 92bf73addb2c)<br>
  you can find it as prefix to the bash window you opened to create the dump; e.g. root@92bf73addb2c:/var/www/html#
- to list all container and ids use:
    > docker ps

#### Import:


copy wikibase.xml to the root of the wikibase container.

> docker cp wikibase.xml "insertcontainernameorid":/var/www/html/

Enable Entity import by changing the localSettings file:

- connect to container and start bash:

> docker-compose exec -it raisewikibase_wikibase_1 bash

- add "$wgWBRepoSettings['allowEntityImport'] = true;" to LocalSettings.php

> echo "$wgWBRepoSettings['allowEntityImport'] = true;" >>> LocalSettings.php

- to check if the line was appended run:

> cat LocalSettings.php

Start the import process by running following comands one after another:

> php maintenance/importDump.php < ../wikibase.xml

> php maintenance/rebuildall.php

> php maintenance/runJobs.php

> php maintenance/initSiteStats.php --update

To finish up you need to run the following script inside your wikibase or you won't be able to manually create new items: https://gist.github.com/JeroenDeDauw/c86a5ab7e2771301eb506b246f1af7a6

- Download the script or copy the lines below to a file named "rebuildWikibaseIdCounters.sql"

- copy it into you wikibase container:

> docker cp rebuildWikibaseIdCounters.sql "insertcontainernameorid":/var/www/html/maintenance

- connect to the container and run the script:

> docker-compose exec -it raisewikibase_wikibase_1 bash

> php maintenance/sql.php rebuildWikibaseIdCounters.sql




## Use dictionaries to populate your Wikibase

The dictionaries have a name of a person or work as key and a list of references as values, e.g.:

{<br>
personname:<br>
             [Sigle_Date_PAGE, Sigle_Date_PAGE, Sigle_Date_PAGE],<br>

personname:<br>
             [Sigle_Date_PAGE, Sigle_Date_PAGE, Sigle_Date_PAGE,]<br>

             ...<br>
}

The following populates the local Wikibase instance step by step:
 1. creates properties and qualifier as a layout for the datamodel used.
 2. creates placeholder to later
     - expand the data model
     - add more data of already existing types (e.g. if you find more person or work names) within a pre defined range of QIDs
 3. creates items for every name found in the dictionaries mentioned above
 4. creates an dictionary for every *Sigle* found in the dictionaries mentioned above
 4. links every them to the *Siglen*
 5. links Theaterperiodika *Siglen* to all their manifestations (e.g. links ABB to ABB 1782, ABB 1785...)
 

## Some pre-processing steps

### create Dictionaries for "Siglen"

Creates two dictionaries as preparation to ingest "Siglen" into the wikibase.

1. {Sigle:description,...}
2. {Sigle:[(SigleWithDate,),...]}


## Helper functions

In [10]:
import regex as re
import pandas as pd
from RaiseWikibase.dbconnection import DBConnection

def getLastID():
    connection = DBConnection()
    lastID = connection.get_last_eid(content_model='wikibase-item')
    connection.conn.close()    
    return lastID

def getSiglenList(pathToList):
    siglenList = pd.read_csv(pathToList, sep="\t", header=None)
    siglenList = siglenList[0].tolist()
    return siglenList

def getSiglenDesc(pathToList):
    siglenTable = pd.read_csv(pathToList, sep="\t", header=None)
    siglenList = siglenTable[0].tolist()
    siglenAufloesung = siglenTable[1].tolist()
    return dict(zip(siglenList,siglenAufloesung))    

def checkDictForSigle(sigle,dictwithSigle): 
    pattern = re.compile(sigle+" \d\d\d\d")
    referencesOfSigle=[]
    for key, value in dictwithSigle.items():
        for entry in value:
            match=re.match(pattern,entry)
            if match and match.group(0) not in referencesOfSigle:
                referencesOfSigle.append(match.group(0))
    
    return referencesOfSigle
                
            
def createDict2(siglenlist, persondict, workdict,schlagwortdict):
    dict2={}
    personSigledict={}
    workSigledict={}
    schlagwortSigledict={}
    for sigle in siglenlist:
        
        personSigleList = checkDictForSigle(sigle,persondict)
        personSigledict[sigle] = personSigleList
        workSigleList = checkDictForSigle(sigle,workdict)
        workSigledict[sigle] = workSigleList
        schlagwortSigleList = checkDictForSigle(sigle,schlagwortdict)
        schlagwortSigledict[sigle] = schlagwortSigleList
        combinedList = sorted(list(set((*personSigleList,*workSigleList,*schlagwortSigleList))))
        #combinedandSortedList = combinedList.sort()
        #dict2[sigle]=combinedandSortedList
        dict2[sigle] = combinedList
    return personSigledict,workSigledict,schlagwortSigledict,dict2


def sortDict(dictToSort):
    sorted_dict_temp = sorted(dictToSort.items(), key=lambda item: item[1])

    sorted_dict = {k: v for k, v in sorted_dict_temp}
    return sorted_dict

def createPlaceHolderList(startID,endID):
    from RaiseWikibase.datamodel import label, alias, description, snak, claim, entity
    from RaiseWikibase.raiser import batch
    #endID included
    i = startID
    placeholderList=[]
    for qID in range(startID,endID+1):
        e = entity(labels=label(value='This is placeholder Q'+str(qID)),
           aliases={},
           descriptions={},
           claims={},
           etype='item')
        
        placeholderList.append(e)
    return(placeholderList)

def createItems(listOfItemTitles):
    itemsToAddList=[]
    for itemTitle in listOfItemTitles:
        itemtmp = entity(labels=label(value=itemTitle),
           aliases={},
           descriptions={},
           claims={},
           etype='item')
        itemsToAddList.append(itemtmp)
    return itemsToAddList


def createProperties(listOfPropertyTitles):
    propertyToAddList=[]
    for propertyTitle in listOfPropertyTitles:
        
        propertytmp = entity(labels=label(value=propertyTitle),  
                           aliases={},
                           descriptions={},
                           claims={},
                           etype='property',
                           datatype='wikibase-property')
        propertyToAddList.append(propertytmp)
    return propertyToAddList        

def dumpjson(pathname, benderDict):
    with open(pathname+".json","w", encoding='utf-8') as jsonfile:
        json.dump(benderDict,jsonfile, ensure_ascii=False)
        print("dumped: "+pathname+".json" )

### Create the Property "subtype of", and others...

To change self defined properties to Wikidata-properties, use WikidataIntegrator + batch (vgl RaiseWikibase/Readme)

In [3]:
from RaiseWikibase.datamodel import label, alias, description, snak, claim, entity
from RaiseWikibase.raiser import batch

listOfPropertyTitles = ['instance of', 'subtype of', 'genre', 'author', 'described by source', 'stated in', 'field of work','sex or gender']

propertyList= createProperties(listOfPropertyTitles)

In [4]:
batch('wikibase-property', propertyList, new=True)

8it [00:00, 31.79it/s]


### Create describing Qualifiers (human, work etc.)

In [5]:
listOfItemTitles=['human', 'work', 'Theaterperiodikum','female','schlagwort']


itemList = createItems(listOfItemTitles)

In [6]:
batch('wikibase-item',itemList)

5it [00:00, 29.83it/s]


#### Placeholders

In [7]:
placeholderList = createPlaceHolderList(6,100)

batch('wikibase-item',placeholderList)

95it [00:02, 36.03it/s]


### Create an item for each "Theaterperiodikum"
also creates a dictionary to later map the subclasses of the Thaterperiodika to their "parents".

dicttmp is a list with 3 dictionaries: 
1. a mapping of sigle: Sigle_With_Dates from the persons dictionary
2. a mapping of sigle: Sigle_With_Dates from the works dictionary
3. a mapping of sigle: Sigle_With_Dates of the combined above

In [9]:
import json

siglenlist= getSiglenList("../data/Tabellen/Siglen_Aufloesung_Bender.tsv")

with open ("../data/txt/Ergebnisse_OCR4all/finished/DramWerkregister_OCR4all.json") as f:
    workfile = json.load(f)
    
with open ("../data/txt/Ergebnisse_OCR4all/finished/Gesamtregister_Personen_checked.json") as f:
    personfile = json.load(f)

with open ("../data/txt/Ergebnisse_OCR4all/finished/Gesamtregister_Schlagworte.json") as f:
    schlagwortefile = json.load(f)

dicttmp = createDict2(siglenlist,personfile,workfile,schlagwortefile)

#siglenDescDict is a mapping of Sigle:Sigle description
siglenDescDict= getSiglenDesc("../data/Tabellen/Siglen_Aufloesung_Bender.tsv")
sorted_dict2 = sortDict(dicttmp[3])

In [13]:
sorted_dict2
dumpjson('../data/txt/zwischenergebnisse/referenceDict', sorted_dict2)

dumped: ../data/txt/zwischenergebnisse/referenceDict.json


In [14]:
#adds Theaterperiodika (e.g "ELB", "RUS") as items
siglenEntities=[]
lastID = getLastID()
i = lastID + 1
sigleQID={} #stores Siglen:ID for referencing later

for key in sorted_dict2:
    e = entity(labels=label(value=key),
               aliases={},
               descriptions=description(value=siglenDescDict[key]),
               claims=claim(prop='P1',
                       mainsnak=snak(datatype='wikibase-item',
                                     value='Q3', #item "Theaterperiodikum"
                                     prop='P1',
                                     snaktype='value')),
               etype='item')
    sigleQID[key]='Q'+str(i)
    i+=1
    siglenEntities.append(e)
#siglenEntities   




### Dump the mapping sigle:QID

In [20]:

        
dumpjson("../data/txt/zwischenergebnisse/siglen_QID_23122021",sigleQID)

dumped: ../data/txt/zwischenergebnisse/siglen_QID_23122021.json


### upload

In [42]:
batch('wikibase-item', siglenEntities, new=True)

153it [00:06, 24.27it/s]


#### placeholder

In [17]:
startID = getLastID()+1
placeholderList = createPlaceHolderList(startID,500)

batch('wikibase-item',placeholderList)

247it [00:06, 36.76it/s]


### Creates entities for every published "Theaterperiodikum" and links them to their "parents"

In [18]:
#adds Theaterperiodika with dates(e.g "ELB 1800", "RUS 1755") as items
siglenWithDatesEntities=[]
lastID = getLastID()
i = lastID + 1

sigleWithDatesQID={} #stores Siglen:ID for referencing later
for key, value in sorted_dict2.items():
    for sigleWithDate in value:
        if sigleWithDate:
            e = entity(labels=label(value=sigleWithDate),
                       aliases={},
                       descriptions=description(value=siglenDescDict[key]),
                       claims=claim(prop='P2',
                               mainsnak=snak(datatype='wikibase-item',
                                             value=sigleQID[key],
                                             prop='P2',
                                             snaktype='value')),
                       etype='item')
            sigleWithDatesQID[sigleWithDate]='Q'+str(i)
            i+=1
            siglenWithDatesEntities.append(e)


In [19]:
def dumpjson(pathname, benderDict):
    with open(pathname+".json","w", encoding='utf-8') as jsonfile:
        json.dump(benderDict,jsonfile, ensure_ascii=False)
        print("dumped: "+pathname+".json" )
        
dumpjson("../data/txt/zwischenergebnisse/siglenWithDates_QID_23122021",sigleWithDatesQID)


dumped: ../data/txt/zwischenergebnisse/siglenWithDates_QID_23122021.json


### upload

In [21]:
batch('wikibase-item', siglenWithDatesEntities, new=True)

265it [00:08, 31.38it/s]


#### placeholder

In [22]:
startID = getLastID()+1
placeholderList = createPlaceHolderList(startID,2000)

batch('wikibase-item',placeholderList)

1235it [00:40, 30.68it/s]


## Prepare dicts

In [23]:
import json
with open ("../data/txt/Ergebnisse_OCR4all/finished/DramWerkregister_OCR4all.json") as f:
    workfile = json.load(f)
    
with open ("../data/txt/Ergebnisse_OCR4all/finished/Gesamtregister_Personen_checked.json") as f:
    personfile = json.load(f)

with open ("../data/txt/Ergebnisse_OCR4all/finished/Gesamtregister_Personen_isFemale.json.json") as f:
    personIsFemaleFile = json.load(f)
with open ("../data/txt/Ergebnisse_OCR4all/finished/Gesamtregister_Schlagworte.json") as f:
    schlagwortefile = json.load(f)

### Persons
adds all Persons from Bender's person name register and links them to all Theaterperiodika (sigle with Dates) they appear in.

In [24]:
dictToLinkPerson =[]
sigleToSiglenListPersons = sortDict(dicttmp[0])
femaleNameList = list(personIsFemaleFile.keys())
for personname, referenceList in personfile.items():
    claims={}
    for reference in referenceList:
        for sigle, sigleListWithDates in sigleToSiglenListPersons.items():
            for sigleWithDate in sigleListWithDates:
                if sigleWithDate in reference:
                    if not claims:
                        claims=claim(prop='P6',
                               mainsnak=snak(datatype='wikibase-item',
                                             value=sigleWithDatesQID[sigleWithDate],
                                             prop='P6',
                                             snaktype='value'))
                    
                    else:
                        tmpclaim=claim(prop='P6',
                               mainsnak=snak(datatype='wikibase-item',
                                             value=sigleWithDatesQID[sigleWithDate],
                                             prop='P6',
                                             snaktype='value'))
                                       
                        claims['P6'].extend(tmpclaim['P6'])
    if personname in femaleNameList:
        claimIsFemale = claim(prop='P8',
                                     mainsnak=snak(datatype='wikibase-item',
                                                   value='Q4', #Q.. for female
                                                   prop='P8', #P... sex or gender
                                                   snaktype='value'))
        claims = claims | claimIsFemale
        
                        
    claimIsHuman = claim(prop='P1',
                                 mainsnak=snak(datatype='wikibase-item',
                                               value='Q1', #Q.. for human
                                               prop='P1',
                                               snaktype='value'))                
                    
    claims = claims | claimIsHuman
    
    e = entity(labels=label(value=personname),
                aliases={},
                descriptions={},
                claims=claims,
                etype='item')
    #e["id"]="Q"+str(i)
                    
    dictToLinkPerson.append(e)
    #i+=1


### upload

In [25]:
batch('wikibase-item', dictToLinkPerson, new=True)

24495it [17:21, 23.51it/s]


#### placeholder

In [26]:
startID = getLastID()+1
placeholderList = createPlaceHolderList(startID,30000)

batch('wikibase-item',placeholderList)

3505it [02:17, 25.41it/s]


### Works
adds all Works from Bender's dramatic work register and links them to all Theaterperiodika (sigle with Dates) they appear in.

In [27]:
dictToLinkWorks = []
sigleToSiglenListWorks = sortDict(dicttmp[1])
for workname, referenceList in workfile.items():
    claims={}
    for reference in referenceList:
        for sigle, sigleListWithDates in sigleToSiglenListWorks.items():
            for sigleWithDate in sigleListWithDates:
                if sigleWithDate in reference:
                    if not claims:
                        claims=claim(prop='P6',
                               mainsnak=snak(datatype='wikibase-item',
                                             value=sigleWithDatesQID[sigleWithDate],
                                             prop='P6',
                                             snaktype='value'))
                    
                    else:
                        tmpclaim=claim(prop='P6',
                               mainsnak=snak(datatype='wikibase-item',
                                             value=sigleWithDatesQID[sigleWithDate],
                                             prop='P6',
                                             snaktype='value'))
                                       
                        claims['P6'].extend(tmpclaim['P6'])
                    
                    
                    
    #add claim workname is instance of work
    claimIsWork = claim(prop='P1',
                                 mainsnak=snak(datatype='wikibase-item',
                                               value='Q2',#value = Q... for work
                                               prop='P1',
                                               snaktype='value'))
    
    
    claims = claims | claimIsWork
    #create entity
    e = entity(labels=label(value=workname),
                aliases={},
                descriptions={},
                claims=claims,
                etype='item')
    #e["id"]="Q"+str(i)
                    
    dictToLinkWorks.append(e)
    #i+=1
                    
#56412 ABB 1776                

### upload

In [28]:
batch('wikibase-item', dictToLinkWorks, new=True)

31760it [27:46, 19.06it/s]


In [ ]:
startID = getLastID()+1
placeholderList = createPlaceHolderList(startID,70000)

batch('wikibase-item',placeholderList)

### Schlagworte
adds all 'schlagworte' from Bender's Schlagwortregister and links them to all Theaterperiodika (sigle with Dates) they appear in.

In [30]:
dictToLinkSchlagworte = []
sigleToSiglenListSchlagworte = sortDict(dicttmp[2])
for schlagwort, referenceList in schlagwortefile.items():
    claims={}
    for reference in referenceList:
        for sigle, sigleListWithDates in sigleToSiglenListSchlagworte.items():
            for sigleWithDate in sigleListWithDates:
                if sigleWithDate in reference:
                    if not claims:
                        claims=claim(prop='P6',
                               mainsnak=snak(datatype='wikibase-item',
                                             value=sigleWithDatesQID[sigleWithDate],
                                             prop='P6',
                                             snaktype='value'))
                    
                    else:
                        tmpclaim=claim(prop='P6',
                               mainsnak=snak(datatype='wikibase-item',
                                             value=sigleWithDatesQID[sigleWithDate],
                                             prop='P6',
                                             snaktype='value'))
                                       
                        claims['P6'].extend(tmpclaim['P6'])
                    
                    
                    
    #add claim schlagwort is instance of schlagwort
    claimIsSchlagwort = claim(prop='P1',
                                 mainsnak=snak(datatype='wikibase-item',
                                               value='Q5',#value = Q... for work
                                               prop='P1',
                                               snaktype='value'))
    
    
    claims = claims | claimIsSchlagwort
    #create entity
    e = entity(labels=label(value=schlagwort),
                aliases={},
                descriptions={},
                claims=claims,
                etype='item')
    #e["id"]="Q"+str(i)
                    
    dictToLinkSchlagworte.append(e)
    #i+=1
                    
#56412 ABB 1776                

In [31]:
batch('wikibase-item', dictToLinkSchlagworte, new=True)

829it [01:24,  9.76it/s]


## Examples of viable SPARQL-Queries

#### sucht alle Ausgaben aller Theaterperiodika heraus
diese Liste ist hilfreich um speziellere Suchanfragen zu formulieren, da auch die IDs der einzelnen Items mit angezeigt wird

#### fügt man ein #defaultView:Tree hinzu bekommt man eine aufklappbare Tabelle

#### sucht alle Ausgaben des Theaterperiodikums ATB

#### sucht alle Personen, die in einer Ausgabe von ATB (Q107) zu finden ist.

#### 